<a href="https://colab.research.google.com/github/jiminmini/mini/blob/main/09_05_%EC%84%B8%EC%85%98_%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [4]:
# import package
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [8]:
# Try GridSearch to optimize hyperparameter
# 1. 일부 데이터만 사용 (예: 10000개)
X_train_small = X_train[:10000]
y_train_small = y_train[:10000]

# 2. GridSearch 하이퍼파라미터 설정
param_grid = {
    'n_neighbors': [3, 4, 5],
    'weights': ['uniform', 'distance']
}

knn_clf = KNeighborsClassifier()

# 3. GridSearchCV (cv=2, 일부 데이터)
grid_search = GridSearchCV(
    knn_clf,
    param_grid,
    cv=2,       # cv 줄이기
    verbose=2,
    n_jobs=-1
)

# 4. 학습
grid_search.fit(X_train_small, y_train_small)


Fitting 2 folds for each of 6 candidates, totalling 12 fits


GridSearchCV(cv=2, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 4, 5],
                         'weights': ['uniform', 'distance']},
             verbose=2)

In [9]:
# best hyperparameter
print("Best hyperparameter:", grid_search.best_params_)

Best hyperparameter: {'n_neighbors': 4, 'weights': 'distance'}


In [10]:
# best score
print("Best CV Score:", grid_search.best_score_)

Best CV Score: 0.9331


In [11]:
# model test
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_acc)

Test Accuracy: 0.9505


### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [12]:
import numpy as np

def shift_image(image, direction):
    """
    MNIST 이미지(1D 배열 또는 2D 배열)를 한 픽셀 이동
    direction: 'left', 'right', 'up', 'down'
    """
    if image.ndim == 1:
        image = image.reshape(28, 28)

    shifted = np.zeros_like(image)

    if direction == 'left':
        shifted[:, :-1] = image[:, 1:]
    elif direction == 'right':
        shifted[:, 1:] = image[:, :-1]
    elif direction == 'up':
        shifted[:-1, :] = image[1:, :]
    elif direction == 'down':
        shifted[1:, :] = image[:-1, :]
    else:
        raise ValueError("direction should be 'left', 'right', 'up', or 'down'")

    return shifted.reshape(-1)  # 다시 1D 벡터로 변환 (MNIST 형식 유지)


In [13]:
# 첫 번째 MNIST 이미지 불러오기
some_digit = X_train[0]

# 이동된 이미지 생성
shifted_left  = shift_image(some_digit, "left")
shifted_right = shift_image(some_digit, "right")
shifted_up    = shift_image(some_digit, "up")
shifted_down  = shift_image(some_digit, "down")

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [14]:
import numpy as np

# 기존 훈련 세트
X_train_orig, y_train_orig = X_train.copy(), y_train.copy()

# 이동 방향 리스트
directions = ['left', 'right', 'up', 'down']

# 새 훈련 세트를 담을 리스트 초기화
X_augmented = []
y_augmented = []

# 모든 이미지에 대해 이동 복사본 생성
for i in range(len(X_train_orig)):
    img = X_train_orig[i]
    label = y_train_orig[i]

    # 원본 이미지도 포함
    X_augmented.append(img)
    y_augmented.append(label)

    # 네 방향으로 이동한 이미지 추가
    for direction in directions:
        shifted_img = shift_image(img, direction)
        X_augmented.append(shifted_img)
        y_augmented.append(label)

In [15]:
# 리스트를 배열로 변환
X_train_aug = np.array(X_augmented)
y_train_aug = np.array(y_augmented)

print("Original train set size:", len(X_train_orig))
print("Augmented train set size:", len(X_train_aug))

Original train set size: 60000
Augmented train set size: 300000


####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [16]:
# 1번 문제에서 GridSearchCV로 찾은 최적 파라미터 사용
best_params = grid_search.best_params_  # 예: {'n_neighbors': 3, 'weights': 'distance'}

# KNN 모델 초기화
knn_best = KNeighborsClassifier(
    n_neighbors=best_params['n_neighbors'],
    weights=best_params['weights'],
    n_jobs=-1
)

In [17]:
# 증식된 데이터로 학습
knn_best.fit(X_train_aug, y_train_aug)

# 테스트 세트 예측
y_test_pred = knn_best.predict(X_test)

In [18]:
# 테스트 정확도 측정
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy with augmented data:", test_accuracy)

Test Accuracy with augmented data: 0.9763
